### Experimento básico de fine-tuning do LLaVa

Autenticação

In [4]:
from os.path import join

KAGGLE_CREDENTIALS_PATH = join('..' ,'.secrets', 'kaggle.json')
DATA_PATH = join('..', 'data')

Download da tabela

In [13]:
from os import environ
from json import load, dump
from zipfile import ZipFile

import pandas as pd

with open(KAGGLE_CREDENTIALS_PATH, 'r', encoding='utf-8') as file:
    json = load(file)
    environ['KAGGLE_USERNAME'] = json['username']
    environ['KAGGLE_KEY'] = json['key']

import kaggle

kaggle.api.authenticate()
kaggle.api.competition_download_file('siim-isic-melanoma-classification',
                                     'train.csv',
                                     join(DATA_PATH, 'compressed'))

with ZipFile(join(DATA_PATH, 'compressed', 'train.csv.zip'), 'r') as file:
    file.extractall(join(DATA_PATH, 'extracted'))

training_data = pd.read_csv(join(DATA_PATH, 'extracted', 'train.csv'))

train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


Download das imagens

In [ ]:
for _, row in training_data.iterrows():
    image_name = f'{row['image_name']}.jpg'
    kaggle.api.competition_download_file('siim-isic-melanoma-classification',
                                         join('jpeg', 'train', image_name),
                                         join(DATA_PATH, 'compressed', 'original_images'))

Extração das imagens

In [36]:
from os import listdir
from shutil import copy

for file in listdir(join(DATA_PATH, 'compressed', 'original_images')):
    if file.endswith('.zip'):
        with ZipFile(join(DATA_PATH, 'compressed', 'original_images', file), 'r') as compressed_file:
            compressed_file.extractall(join(DATA_PATH, 'extracted', 'original_images'))
    else:
        copy(join(DATA_PATH, 'compressed', 'original_images', file), join(DATA_PATH, 'extracted', 'original_images', file))

Redefinição de tamanho para no máximo 1000p

In [38]:
from multiprocessing import Pool
from PIL import Image

def resize(file):
    with Image.open(join(DATA_PATH, 'extracted', 'original_images', file)) as image:
        new_size_proportion = 1000 / max(image.size)
        resized_image = image.resize((int(image.size[0] * new_size_proportion), int(image.size[1] * new_size_proportion)))
        resized_image.save(join(DATA_PATH, 'images_0', file), 'JPEG')

with Pool() as pool:
    pool.map(resize, listdir(join(DATA_PATH, 'extracted', 'original_images')))

Definição dos dados de treinamento

In [31]:
from uuid import uuid4

dataset = []

for _, row in training_data.iterrows():
    dataset.append({
        'id': str(uuid4()),
        'image': f'{row['image_name']}.jpg',
        'conversations': [
            {
                'from': 'human',
                'value': 'Is this a melanoma benign or malignant?'
            },
            {
                'from': 'gpt',
                'value': f'This melanoma is {row['benign_malignant']}'
            }
        ]
    })

with open(join(DATA_PATH, 'training_dataset.json'), 'w', encoding='utf-8') as file:
    dump(dataset, file, ensure_ascii=False, indent=4)